In [8]:
import matplotlib.pyplot as plt
%matplotlib widget

import os
import pointCollection as pc
import h5py
import numpy as np
import scipy.ndimage as snd
import h5py

In [ ]:
! h5ls ../../../ice-shelf-geometry/data/icemask_composite.nc

In [ ]:
! ls 

In [ ]:
mask_file='../../../ice-shelf-geometry/data/icemask_composite.nc'
template=pc.grid.data(t_axis=0).from_nc(mask_file, bands=[18], fields=[])

files=['../../../ice-shelf-geometry/data/icemask_composite.nc',
       'antarctic_icelines_2021_mask.h5',
       'antarctic_icelines_2022_mask.h5']
 
t_vals=[]
file_nums=[]

for count, file in enumerate(files):
    with h5py.File(file,'r') as h5f:
        if 't' in h5f:
            t_temp = np.array(h5f['t'])
        else:
            t_temp=np.array(h5f['year'])
        keep = t_temp > 2000
        t_vals += list(t_temp[keep])
        file_nums += list(np.zeros(keep.sum(), dtype=int)+count)

t_vals=np.array(t_vals)
        
# remove the last of the original Greene et al masks
t_vals=t_vals[np.flatnonzero(np.round(t_vals/.1)*.1 != 2021.2)]

In [ ]:
# read the bedmachine mask and ice thickness
bedmachine_file='/Volumes/ice3/ben/Bedmachine/BedMachineAntarctica_2020-07-15_v02.nc'
temp=pc.grid.data().from_nc(bedmachine_file, fields=['mask','thickness'])#, bounds=[[3.e5, 5.e5], [-14e5, -12e5]])

# dilate the grounded-ice and ice-shelf parts of the mask by 4 pixels
temp.assign({'z':(snd.binary_dilation((temp.mask==1)|(temp.mask==2), structure=np.ones((4,4), dtype=bool)) & (temp.thickness < 10))})
bm_mask=pc.grid.data().from_dict({'x':template.x,'y':template.y})

# interpolate the bedmachine mask to the Green coordinates
bm_mask.assign({'z':temp.interp(template.x, template.y, gridded=True)<0.9})
bm_mask.z = snd.binary_opening(bm_mask.z, np.ones([10,10], dtype='bool'))

# mask the pole hole
bm_mask.get_latlon(srs_epsg=3031)
bm_mask.z[bm_mask.latitude < -88]=0


In [ ]:
out_file='Greene_22_shelf_plus_10m_mask_full.h5'

fields=['x','y','year']


if True:

    #if not os.path.isfile(in_file):
    #    ! pushd ../../..; git clone git@github.com:SmithB/ice-shelf-geometry.git

    # get the dimension shapes:
    D={'x':template.x,'y':template.y,'year':np.array(t_vals)}

    if os.path.isfile(out_file):
        os.remove(out_file)
    with h5py.File(out_file,'w') as h5f_out:
        h5f_out.create_dataset('z', [D['y'].size, D['x'].size, D['year'].size], 
                               chunks=(100, 100, 1), compression='gzip', dtype='i8', \
                              fillvalue=255)
        h5f_out.create_dataset('x', data=template.x)
        h5f_out.create_dataset('y', data=template.y)
        h5f_out.create_dataset('t', data=t_vals)
        for count, in_file in enumerate(files):
            with h5py.File(in_file,'r') as h5f_in:
                if 't' in h5f_in:
                    t_temp = np.array(h5f_in['t'])
                else:
                    t_temp=np.array(h5f_in['year'])
                for this_field in ['z','ice']:
                    if this_field in h5f_in:
                        break
            _, in_bands, out_bands=np.intersect1d(t_temp, t_vals, return_indices=True)
            for in_band, out_band in zip(in_bands, out_bands):
                if in_file[-2:]=='h5':
                    in_mask=pc.grid.data().from_h5(in_file, bands=[in_band], field_mapping={'z':this_field})
                else:
                    in_mask=pc.grid.data().from_nc(in_file, bands=[in_band], field_mapping={'z':this_field}, timename='year')
                print(f"writing input band {in_band} to output band {out_band}")
                # transpose each band, flip in y
                h5f_out['z'][:,:,out_band] = (np.squeeze(in_mask.z==1) & bm_mask.z)

In [ ]:
np.diff(t_vals)

## Make the 1-km mask used to generate the tile centers

In [ ]:
mask=pc.grid.data().from_h5(out_file, bands=[0])
mask.z=mask.z.astype(bool)
mask.to_geotif('Greene_22_shelf_plus_10m_mask_240m.tif', srs_epsg=3031)
! rm Greene_22_shelf_plus_10m_mask_1km.tif
! gdal_translate -tr 1000 1000 -r nearest -co COMPRESS=LZW -co TILED=YES -co PREDICTOR=1 Greene_22_shelf_plus_10m_mask_240m.tif Greene_22_shelf_plus_10m_mask_1km.tif


In [ ]:
M_1km=pc.grid.data().from_geotif('Greene_22_shelf_plus_10m_mask_1km.tif')
M_1km.show()

In [ ]:
#XR, YR=np.round(np.array([*map(np.array, [plt.gca().get_xlim(), plt.gca().get_ylim()])])/1.e4)*10000
XR, YR =(np.array([-1660000., -1490000.]), np.array([-360000., -230000.]))


In [ ]:
D=pc.grid.data().from_h5('Greene_22_shelf_plus_10m_mask_full.h5', bounds=[XR, YR])

In [ ]:
import matplotlib.animation as animation
import glob
plt.rcParams['animation.ffmpeg_path'] ='/home/ben/mambaforge/envs/devel/bin/ffmpeg'
%matplotlib widget

In [ ]:

def draw_mask(k0, D, hi, ha):
    
    k=np.minimum(D.t.size-1, np.maximum(0, k0-pause_frames))
    hi.set_data((D.z[:,:,k]))
    ht=ha.set_title(f'{D.t[k]:2.2f}')
    return [hi, ht]


hf, ha=plt.subplots(1,1)
hi=D.show(band=0, ax=ha)
ha.set_title(f'{D.t[0]:2.2f}')

pause_frames=1
anim = animation.FuncAnimation(hf, draw_mask, frames=D.z.shape[2]+2*pause_frames, interval=500, blit=True, fargs=[D, hi, ha])
from matplotlib import rc

# equivalent to rcParams['animation.html'] = 'html5'
rc('animation', html='html5')
anim
